In [ ]:
from collections import defaultdict
from typing import List, Set

# ----------  Estructuras y utilidades ----------

class Edge:
    def __init__(self, start: str, end: str):
        self.start = start
        self.end = end

def print_vector(vec: List[str]):
    print("[" + ", ".join(sorted(map(str, vec))) + "]", end="")

def print_2D_vector(vecs: List[List[str]]):
    print("[", end="")
    for i in range(len(vecs) - 1):
        print_vector(vecs[i])
        print(", ", end="")
    print_vector(vecs[-1])
    print("]")

# ----------  Variables globales ----------

cliques: List[List[str]] = []      # lista de cliques maximales
backtrack_count: int = 0           # nº de backtracks (P=∅, X≠∅)

# ----------  Algoritmo Bron–Kerbosch + pivoting ----------

def bron_kerbosch(current_clique: Set[str],
                  candidates: Set[str],
                  processed_vertices: Set[str],
                  graph: defaultdict[str, Set[str]]) -> None:
    """
    Versión con pivoting + contador de backtracks.
    """
    global cliques, backtrack_count

    # 1) Caso base: no candidatos NI excluidos  → clique maximal
    if not candidates and not processed_vertices:
        if len(current_clique) > 2:              # sólo si interesa cliques ≥3
            cliques.append(list(current_clique))
        return

    # 2) Caso base: no candidatos PERO sí excluidos → backtracking
    if not candidates and processed_vertices:
        backtrack_count += 1
        print(f"Backtrack #{backtrack_count}: R={sorted(current_clique)}  "
              f"(X tiene {len(processed_vertices)} vértices)")
        return

    # 3) Elegir pivote u con mayor |P ∩ N(u)|
    union_set = candidates.union(processed_vertices)
    pivot = max(union_set, key=lambda v: len(graph[v]))

    # 4) Vertices a explorar: P \ N(u)
    possibles = candidates.difference(graph[pivot])

    for vertex in list(possibles):   # list() porque modificaremos 'candidates'
        new_clique = current_clique.union({vertex})
        new_candidates = candidates.intersection(graph[vertex])
        new_processed_vertices = processed_vertices.intersection(graph[vertex])

        # Recursión
        bron_kerbosch(new_clique, new_candidates,
                      new_processed_vertices, graph)

        # 5) Mover 'vertex' de P a X  (líneas 8–9 del pseudocódigo)
        candidates.remove(vertex)
        processed_vertices.add(vertex)

# ----------  Programa principal ----------

def main():
    global cliques, backtrack_count

    # --- Grafo de ejemplo: triángulo (a,b,c) + cuadrado abierto (d,e,f,g) ---
    # ---------------------------------------------------------------------
#  Lista de aristas SIN duplicados, usando objetos Edge
# ---------------------------------------------------------------------
    edges = [
        # Grafo no-dirigido (con aristas en ambas direcciones)
        Edge("A1", "B1"), Edge("B1", "A1"),
        Edge("A1", "C1"), Edge("C1", "A1"),
        Edge("A1", "D1"), Edge("D1", "A1"),
        Edge("A1", "E1"), Edge("E1", "A1"),
        Edge("A1", "F1"), Edge("F1", "A1"),

        Edge("B1", "C1"), Edge("C1", "B1"),
        Edge("B1", "D1"), Edge("D1", "B1"),
        Edge("C1", "D1"), Edge("D1", "C1"),
        Edge("E1", "F1"), Edge("F1", "E1"),



        Edge("E1", "A2"), Edge("A2", "E1"),
        Edge("A2", "F2"), Edge("F2", "A2"),
        Edge("F2", "E2"), Edge("E2", "F2"),
        Edge("A2", "E2"), Edge("E2", "A2"),



        Edge("A2", "D2"), Edge("D2", "A2"),
        Edge("A2", "C2"), Edge("C2", "A2"),
        Edge("A2", "B2"), Edge("B2", "A2"),
        Edge("B2", "C2"), Edge("C2", "B2"),
        Edge("B2", "D2"), Edge("D2", "B2"),
        Edge("D2", "C2"), Edge("C2", "D2"),


        Edge("A3", "B3"), Edge("B3", "A3"),
        Edge("A3", "C3"), Edge("C3", "A3"),
        Edge("A3", "D3"), Edge("D3", "A3"),
        Edge("B3", "C3"), Edge("C3", "B3"),

        Edge("B3", "D3"), Edge("D3", "B3"),
        Edge("D3", "C3"), Edge("C3", "D3"),
        Edge("A3", "E2"), Edge("E2", "A3"),
        Edge("A3", "F3"), Edge("F3", "A3"),


        Edge("A3", "E3"), Edge("E3", "A3"),
        Edge("F3", "E3"), Edge("E3", "F3"),

        Edge("A4", "B4"), Edge("B4", "A4"),
        Edge("A4", "C4"), Edge("C4", "A4"),
        Edge("A4", "D4"), Edge("D4", "A4"),
        Edge("B4", "C4"), Edge("C4", "B4"),
        Edge("B4", "D4"), Edge("D4", "B4"),
        Edge("C4", "D4"), Edge("D4", "C4"),
        Edge("B4", "F1"), Edge("F1", "B4"),


        Edge("A5", "B5"), Edge("B5", "A5"),
        Edge("A5", "C5"), Edge("C5", "A5"),
        Edge("A5", "D5"), Edge("D5", "A5"),
        Edge("B5", "C5"), Edge("C5", "B5"),
        Edge("B5", "D5"), Edge("D5", "B5"),
        Edge("C5", "D5"), Edge("D5", "C5"),
        Edge("E1", "B5"), Edge("B5", "E1"),
        Edge("B5","F2" ), Edge("F2", "B5"),


        Edge("A6", "B6"), Edge("B6", "A6"),
        Edge("A6", "C6"), Edge("C6", "A6"),
        Edge("A6", "D6"), Edge("D6", "A6"),
        Edge("B6", "C6"), Edge("C6", "B6"),
        Edge("B6", "D6"), Edge("D6", "B6"),
        Edge("C6", "D6"), Edge("D6", "C6"),
        Edge("B6", "E2"), Edge("E2", "B6"),
        Edge("B6", "F3"), Edge("F3", "B6"),

        Edge("A7", "B7"), Edge("B7", "A7"),
        Edge("A7", "C7"), Edge("C7", "A7"),
        Edge("A7", "D7"), Edge("D7", "A7"),
        Edge("B7", "C7"), Edge("C7", "B7"),
        Edge("B7", "D7"), Edge("D7", "B7"),
        Edge("C7", "D7"), Edge("D7", "C7"),
        Edge("B7", "E3"), Edge("E3", "B7"),


        Edge("A4", "F4"), Edge("F4", "A4"),
        Edge("F4", "E4"), Edge("E4", "F4"),
        Edge("A4", "E4"), Edge("E4", "A4"),
        Edge("E4", "A5"), Edge("A5", "E4"),

        Edge("A5", "F5"), Edge("F5", "A5"),
        Edge("A5", "E5"), Edge("E5", "A5"),
        Edge("F5", "E5"), Edge("E5", "F5"),
        Edge("E5", "A6"), Edge("A6", "E5"),

        Edge("A6", "F6"), Edge("F6", "A6"),
        Edge("E6", "A6"), Edge("A6", "E6"),
        Edge("F6", "E6"), Edge("E6", "F6"),

        Edge("E6", "A7"), Edge("A7", "E6"),
        Edge("A7", "F7"), Edge("F7", "A7"),
        Edge("A7", "E7"), Edge("E7", "A7"),
        Edge("F7", "E7"), Edge("E7", "F7")


    ]
    # Construir lista de adyacencia
    graph = defaultdict(set)
    for edge in edges:
        graph[edge.start].add(edge.end)

    # Inicializar conjuntos
    current_clique: Set[str] = set()
    candidates:      Set[str] = set(graph.keys())
    processed:       Set[str] = set()

    # Ejecutar algoritmo
    bron_kerbosch(current_clique, candidates, processed, graph)

    # Mostrar resultados
    cliques.sort(key=lambda x: (len(x), x))
    print("\nCliques maximales encontrados: ", end="")
    print_2D_vector(cliques)
    print(f"\nTotal de backtracks contabilizados: {backtrack_count}")
    print(f"Total clicques maximos {len(cliques)}")

if __name__ == "__main__":
    main()
